<table  align="left" width="100%"> <tr>
        <td  style="background-color:#ffffff;"><a href="https://qworld.net" target="_blank"><img src="../images/qworld.jpg" width="35%" align="left"></a></td>
        <td  align="right" style="background-color:#ffffff;vertical-align:bottom;horizontal-align:right">
            prepared by <a href="https://iitis.pl/pl/person/aglos" target="_blank"  >Adam Glos</a>
        </td>        
</tr></table>

# <font color="blue"> Solutions for </font> Grover's Algorithm revisited

<a id="task1"></a>
### Task 1 

Create a 4-qubit circuit named `circuit` and apply a Hadamard gate on each qubit. Convince yourself about the correctness of the circuit by displaying it. Print the quantum state with `Simulator` without measurement.

In [1]:
import cirq

# create the circuit
circuit = cirq.Circuit()
qubits = cirq.LineQubit.range(4)
circuit.append(cirq.H.on_each(*qubits))

# for displaying the circuit
print(circuit)

# to print the state
s = cirq.Simulator()
print(s.simulate(circuit))
print(s.simulate(circuit).dirac_notation())

0: ───H───

1: ───H───

2: ───H───

3: ───H───
measurements: (no measurements)

qubits: (cirq.LineQubit(0),)
output vector: 0.707|0⟩ + 0.707|1⟩

qubits: (cirq.LineQubit(1),)
output vector: 0.707|0⟩ + 0.707|1⟩

qubits: (cirq.LineQubit(2),)
output vector: 0.707|0⟩ + 0.707|1⟩

qubits: (cirq.LineQubit(3),)
output vector: 0.707|0⟩ + 0.707|1⟩

phase:
output vector: |⟩
0.25|0000⟩ + 0.25|0001⟩ + 0.25|0010⟩ + 0.25|0011⟩ + 0.25|0100⟩ + 0.25|0101⟩ + 0.25|0110⟩ + 0.25|0111⟩ + 0.25|1000⟩ + 0.25|1001⟩ + 0.25|1010⟩ + 0.25|1011⟩ + 0.25|1100⟩ + 0.25|1101⟩ + 0.25|1110⟩ + 0.25|1111⟩


<a id="task2"></a>
### Task 2 
    
Create a 3-qubit register and design the Grover diffusion operator based on the circuit above. Print the circuit and its matrix form by using `cirq.unitary`. Verify that the circuit by checking the entries of the matrix.

<h3> Solution </h3>

In [ ]:
import cirq
from cirq import H, X, Z

# create the circuit
qq = cirq.LineQubit.range(3)
circuit = cirq.Circuit()
circuit.append(H.on_each(*qq))
circuit.append(X.on_each(*qq))
circuit.append(Z(qq[2]).controlled_by(qq[0], qq[1]))
circuit.append(X.on_each(*qq))
circuit.append(H.on_each(*qq))

# display the circuit
print(circuit)

# Print the unitary matrix. Note that on the entries on the diagonal should equal 1-2/N = 3/4, 
# while for the outer-diagonal it should be -2/N = 1/4 =0.125

N = 2**3
print("On the diagonal:", 1-2/N)
print("Outside the main diagonal:", -2/N, "\n")
print(cirq.unitary(circuit))

<a id="task3"></a>
### Task 3

Fill the `oracle` and `grover_diffusion` so that they will form a 4-qubit Grover's algorithm. You can use the code from previous tasks. Verify the results by analyzing the statistics of the measurements.

<h3> Solution </h3>

In [ ]:
import cirq
from cirq import H, Z, X

qq = cirq.LineQubit.range(4)
circuit = cirq.Circuit()
circuit.append(H.on_each(*qq))

def oracle():
    yield Z(qq[3]).controlled_by(*(qq[0:3]))


def grover_diffusion():
    yield H.on_each(*qq)
    yield X.on_each(*qq)
    yield Z(qq[3]).controlled_by(*(qq[0:3]))
    yield X.on_each(*qq)
    yield H.on_each(*qq)

for i in range(3):
    circuit.append(oracle())
    circuit.append(grover_diffusion())

circuit.append(cirq.measure(*qq, key='result'))
    
# determine the statistics of the measurements
s = cirq.Simulator() 
trials_number = 1000
samples = s.run(circuit, repetitions=trials_number)

def bitstring(bits):
    return "".join(str(int(b)) for b in bits)

counts = samples.histogram(key="result",fold_func=bitstring)

print("Measurement output: ", counts)
print("Probability of measuring 1111: ", counts.get('1111')/trials_number)
print("Probability of not measuring 1111: ", 1- counts.get('1111')/trials_number)